In [1]:
import numpy as np #数字库
import tensorflow as tf
import matplotlib.pyplot as plt #2D绘图库
from tensorflow.examples.tutorials.mnist import input_data

print("packs loaded")

packs loaded


In [2]:
#下载mnist 查看数据量
print("Download and Extract MNIST dataset")
mnist = input_data.read_data_sets('data/', one_hot=True)
print
print("type of 'mnist' is %s" % (type(mnist)))
print("number of train data is %d" % (mnist.train.num_examples))
print("number of test data is %d" % (mnist.test.num_examples))

Download and Extract MNIST dataset
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use urllib or similar directly.


URLError: <urlopen error [WinError 10060] 由于连接方在一段时间后没有正确答复或连接的主机没有反应，连接尝试失败。>

In [3]:
#设置神经网络的参数
n_hidden_1 = 256 #隐藏层的神经元个数
n_hidden_2 = 128
n_input    = 784 #输入的像素值 像素值是固定值  全连接网络是限定死的 后面的卷积神经网络可变
n_classes  = 10  #分成多少类    固定值

#INPUTS AND OUTPUTS  
x = tf.placeholder("float",[None,n_input]) #placeholder相当于形参 定义一个 数据类型为float none维(这里的none相当于占位符) 名称为n_input的参数
y = tf.placeholder("float",[None,n_classes])

# NETWORK PARAMETERS  这里没有涉及具体的数据 只是搭建出矩阵的框子 具体的数据没有写入
'''
    tf.random_normal函数 用于从服从指定正太分布的数值中取出指定个数的值
    shape: 输出张量的形状，必选
    mean: 正态分布的均值，默认为0
    stddev: 正态分布的标准差，默认为1.0
    dtype: 输出的类型，默认为tf.float32
    seed: 随机数种子，是一个整数，当设置之后，每次生成的随机数都一样
    name: 操作的名称
'''
stddev = 0.1 
weights = {
    'w1':tf.Variable(tf.random_normal([n_input,n_hidden_1],stddev=stddev)), #Variable（）用来构造任何类型和形状的Tensor（张量）。 初始值定义变量的类型和形状。 施工后，变量的类型和形状是固定的
    'w2':tf.Variable(tf.random_normal([n_hidden_1,n_hidden_2],stddev=stddev)),
    'out':tf.Variable(tf.random_normal([n_hidden_2,n_classes],stddev=stddev))
}
#偏置项
biases = {
    'b1':tf.Variable(tf.random_normal([n_hidden_1])),
    'b2':tf.Variable(tf.random_normal([n_hidden_2])),
    'out':tf.Variable(tf.random_normal([n_classes]))
}
print("NETWORK READY")

NETWORK READY


In [4]:
def multilayer_perceptron(_X,_weights,_biases):
    layer_1 = tf.nn.relu(tf.add(tf.matmul(_X,_weights['w1']),_biases['b1']))
    layer_2 = tf.nn.relu(tf.add(tf.matmul(layer_1,_weights['w2']),_biases['b2']))
    return (tf.matmul(layer_2,_weights['out'])+_biases['out'])

In [5]:
# PREDICTION
pred = multilayer_perceptron(x,weights,biases)

#LOSS AND OPTIMIZER 定义最优控制器
'''
tf.argmax(input, axis=None, name=None, dimension=None)

此函数是对矩阵按行或列计算最大值

参数
input：输入Tensor
axis：0表示按列，1表示按行
name：名称
dimension：和axis功能一样，默认axis取值优先。新加的字段
返回：Tensor  一般是行或列的最大值下标向量
'''
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred,labels=y)) #计算出cost的均值 
optm = tf.train.GradientDescentOptimizer(learning_rate=0.01).minimize(cost) #采用梯度下降优化cost值
corr = tf.equal(tf.argmax(pred,1),tf.argmax(y,1)) #判断是否预测对
accr = tf.reduce_mean(tf.cast(corr,"float")) #平均的精度为多少

#INITIALIZER
init = tf.global_variables_initializer()
print("FUNCTIONS READY")

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.

FUNCTIONS READY


In [6]:
training_epochs = 20
batch_size      = 100
display_step    = 4
#LAUNCH THE GRAPH
sess = tf.Session()
sess.run(init)
#OPTIMIZE
for epoch in range(training_epochs):
    avg_cost = 0.
    total_batch = int(mnist.train.num_examples/batch_size)
    # ITERATION
    for i in range(total_batch):
        batch_xs,batch_ys = mnist.train.next_batch(batch_size) #取下一个batch
        feeds = {x:batch_xs, y:batch_ys} 
        sess.run(optm,feed_dict=feeds) # feed_dict的作用是给使用placeholder创建出来的tensor赋值  #运行optm 并给tensor：上面的x y赋值 
        avg_cost += sess.run(cost,feed_dict=feeds)
    avg_cost = avg_cost/total_batch
    
    if(epoch+1)%display_step==0:
        print("Epoch: %03d/%03d cost: %.9f" % (epoch,training_epochs,avg_cost))
        feeds = {x:batch_xs,y:batch_ys}
        train_acc = sess.run(accr,feed_dict=feeds) #此时的fetch = accr
        print("TRAIN ACCURACY: %.3f"%(train_acc))
        feeds = {x:mnist.test.images,y:mnist.test.labels}
        test_acc = sess.run(accr,feed_dict=feeds) #此时的fetch = accr
        print("TEST ACCURACY: %.3f"%(test_acc))
print("OPTIMIZATION FINISHED")

Epoch: 003/020 cost: 0.312826733
TRAIN ACCURACY: 0.910
TEST ACCURACY: 0.905
Epoch: 007/020 cost: 0.240327804
TRAIN ACCURACY: 0.910
TEST ACCURACY: 0.926
Epoch: 011/020 cost: 0.199365704
TRAIN ACCURACY: 0.940
TEST ACCURACY: 0.938
Epoch: 015/020 cost: 0.170049531
TRAIN ACCURACY: 0.950
TEST ACCURACY: 0.945
Epoch: 019/020 cost: 0.147246379
TRAIN ACCURACY: 0.980
TEST ACCURACY: 0.949
OPTIMIZATION FINISHED
